In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
from PIL import Image, ImageStat
from PIL import __version__
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from PIL import Image
# Device configuration
device = torch.device('cuda:0')

In [2]:
#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter('board/fnm_exp1')

#!tensorboard --logdir I:\tensorboard

In [3]:
import os
import numpy as np
#import tensorflow as tf

In [4]:
from math import floor

In [5]:
# use_bias = False
# front_path = './images/fig1'
# profile_path = '.images/fig2'
lambda_l1 = 0.001 #'weight of the loss for L1 texture loss') # 0.001
lambda_fea=100 #'weight of the loss for face model feature loss')
lambda_reg= 1e-5# 'weight of the loss for L2 regularitaion loss')
lambda_gan= 1# 'weight of the loss for gan loss')
lambda_gp=10# 'weight of the loss for gradient penalty on parameter of D')
lambda_mirror = 0.5
# For training
dataset_size=  1000# 'dataset path')  # casia_aligned_250_250_jpg
profile_list=''# 'train profile list')
front_path=''#front data path')
front_list=''# 'train front list')
test_path=''# 'front data path')
is_train=True# 'train or test')
# is_finetune= False# 'finetune') # False, True
# face_mode='resnet50.npy'# 'face model path')
# checkpoint='checkpoint/fnm'# 'checkpoint directory')
# summary_dir= 'log/fnm'# 'logs directory')
# checkpoint_ft='checkpoint/fnm/ck-09'#'finetune or test checkpoint path')
batch_size=4# 'batch size')#was 16
BS = batch_size
epoc=10 # 'epoch')
critic= 1 #'number of D training times')
# save_freq= 1000 # 'the frequency of saving model')
lr=1e-4# 'base learning rate')
beta1=0. # 'beta1 momentum term of adam')
beta2=0.9 # 'beta2 momentum term of adam')
stddev= 0.02 # 'stddev for W initializer')
use_bias=False # 'whether to use bias')
results='results/fnm' # 'path for saving results') #
############################
############################
#   environment setting    #
############################
device_id='3,4'# 'device id')
ori_height=224 # 'original height of profile images')
ori_width=224 # 'original width of profile images')
height= 224 #'height of images') # do not modified
width= 224 # 'width of images') # do not modified
CHANNEL=3 # 'channel of images')
num_threads=8 # 'number of threads of enqueueing examples')


# Resnet50 model trained on VGGFace2 dataset

In [6]:
from Resnet50_ft_dag import resnet50_ft_dag


In [7]:
weights_path= "./resnet50_ft_dag.pth"
Model = resnet50_ft_dag(weights_path=weights_path).to(device)
for p in Model.parameters():
        p.require_grad = False

In [8]:
#writer = SummaryWriter("FNM", comment = "19.03.2020")

# Batch normalisation

In [9]:
# class Batch_norm(nn.Module):
#     def __init__(self, in_channels, epsilon=1e-5, momentum = 0.9, is_train = False):
#         super(Batch_norm, self).__init__()
#         # self.mean  = mosv_dict['mean']
#         self.norm = nn.BatchNorm2d(in_channels, eps = epsilon,  momentum = momentum, track_running_stats = is_train)
#         #self.scale = mosv_dict['scale']
#         #self.variance = mosv_dict['variance']
#         #self.epsilon = 1e-5
#     def forward(self, x):
#         norm = self.norm(x)
#         return norm
            
    

In [10]:
def Batch_norm(in_channels, epsilon=1e-5, momentum = 0.9, is_train = False):
    return nn.BatchNorm2d(in_channels, eps = epsilon,  momentum = momentum, track_running_stats = is_train)

In [11]:
NORM = Batch_norm

# Options: conv2d, res_block 

In [12]:
def calc_conv_pad(input_size, output_size, filter_size = 3, stride = 2):
    return max(0, floor((stride * (output_size - 1) + filter_size - input_size)/2 + 0.5))

In [13]:
def calc_deconv_pad(input_size, output_size, filter_size = 3, stride = 2):
    return max(0, floor((stride * (input_size - 1) + filter_size - output_size)/2 + 0.5))

In [14]:
def conv2d( in_channels , out_channels , kernel_size = 3 , strides = 1  , 
           padding  =  None,  pad_input = None,
           bias = use_bias,dilation_rate = 1, activation = None):
    layers = []
    if padding == None:
        if pad_input is not None:
            padding = calc_conv_pad(pad_input , 2*pad_input, kernel_size , strides)
        else:
            padding = 0
    conv = nn.Conv2d( in_channels , out_channels , bias=bias, kernel_size = kernel_size, stride=strides, padding=padding, dilation=dilation_rate )
    layers.append( conv )
    if activation is not None:
        layers.append( activation )
    return nn.Sequential( *layers )

def deconv2d(in_channels , out_channels , kernel_size = 3 , strides = 1  , 
             padding  = None, pad_input = None,
             bias= use_bias, dilation_rate = 1,
            activation = None):
    if padding == None:
        if pad_input is not None:
            padding = calc_deconv_pad(pad_input , 2*pad_input, kernel_size , strides)
        else:
            padding = 0
    layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size =  kernel_size, stride=strides, 
                      padding=padding, bias=bias)]
    if activation is not None:
        layers.append( activation )
    return nn.Sequential( *layers )
    

In [15]:
class res_block(nn.Module):
    def __init__(self,  in_channels , 
                 out_channels , kernel_size = 3, 
                 stride = 1  , padding  =  None, pad_input = None,
                 bias = use_bias,  norm = NORM, activation2 = nn.ReLU, activation = None,):
        super(res_block, self).__init__()
        if padding == None:
            if pad_input is not None:
                padding = calc_conv_pad(pad_input , pad_input, kernel_size , stride)
            else: 
                padding = 0
        self.out_channels = out_channels
        if activation is not None:
            self.activation = activation(out_channels)
        else:
            self.activation = activation
        self.activation2 = activation2(out_channels)
        convs = [conv2d(in_channels , out_channels , kernel_size , stride , padding, bias, activation = self.activation), 
             nn.BatchNorm2d( in_channels ), 
             nn.ReLU(out_channels), 
             conv2d(out_channels, out_channels, kernel_size, stride, padding, bias, activation = self.activation),
             norm(in_channels)]
        self.layers = nn.Sequential(*convs)
        #независимо от передаваемого параметра NORM норма всегда Batch_norm
    def forward(self, x):
        return self.activation2(self.layers(x) + x)

# Load data

In [16]:
from random import random

In [17]:
import glob
import shutil

profile_set = set()
profile_list = []
front_set = set()
front_list = []
frontFile = open("./TPdata/train_data/front_list.txt", "w")
profileFile = open("./TPdata/train_data/profile_list.txt", "w")

# def relocate_data(src_dir, src_dir_front, src_dir_profile, variety = 3):
#     for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
#         #print(jpgfile);
#         #break;
#         tmp = jpgfile.split('\\')[-1].split('.')[0].split('_')
#         ttmp = tmp[0]+ "_"+tmp[2]
#         if (len(tmp) > 6):
#             if ttmp not in profile_set or random()< 3/8:
#                 shutil.copy(jpgfile, src_dir_profile)
#                 profileFile.write(jpgfile+"\n")
#                 #profile_list.append(jpgfile)
#                 profile_set.add(ttmp)
#         else:
#             if ttmp not in front_set or random()< 3/7:
#                 shutil.copy(jpgfile, src_dir_front)
#                 frontFile.write(jpgfile+"\n")
#                 #front_list.append(jpgfile)
#                front_set.add(ttmp)
def relocate_data(src_dir, src_dir_front, src_dir_profile, variety = 3):
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        #print(jpgfile); 
        #break;
        tmp = jpgfile.split('\\')[-1].split('.')[0].split('_')
        ttmp = tmp[0]+ "_"+tmp[2]
        jpgfile =  jpgfile.split('\\')[0]+"/" +jpgfile.split('\\')[1]
        if (len(tmp) == 6):
            if ttmp not in front_set or random()< 0.8:
            #shutil.copy(jpgfile, src_dir_front)
                frontFile.write(jpgfile+"\n")
                front_set.add(ttmp)
        else:
            if ttmp not in profile_set or random()< 1/5:
                #shutil.copy(jpgfile, src_dir_profile)
                profileFile.write(jpgfile+"\n")
                #profile_list.append(jpgfile)
                profile_set.add(ttmp)
            
            
black_set = set()
def choose_cfp(src_dir, src_dir_front, src_dir_profile):
    for i in range(1, 501):
        curdir = str(i)
        if i<10:
            curdir= "/00"+curdir
        elif i>=10 and i<100:
            curdir = "/0"+curdir
        else:
            curdir = "/"+curdir
        curdirfront = curdir + "/frontal"
        curdirprofile = curdir + "/profile"
        a = 0
        for jpgfile in glob.iglob(os.path.join(src_dir+curdirfront, "*.jpg")):
            jpgfile =  jpgfile.split('\\')[0]+"/" +jpgfile.split('\\')[1]
#             im = Image.open(jpgfile).convert("RGB")
#             stat = ImageStat.Stat(im)
#             if sum(stat.sum)/3==stat.sum[0]:
#                 print(jpgfile)
            if  a==0 and random()< 1/7:
                #shutil.copy(jpgfile, src_dir_front)
                frontFile.write(jpgfile+"\n")
            elif a==1:
                #shutil.copy(jpgfile, src_dir_profile)
                profileFile.write(jpgfile+"\n")
            elif (a < 2) and random()<1/8:
                profileFile.write(jpgfile+"\n")
            else:
                break
            a+= 1
        a = 0
        for jpgfile in glob.iglob(os.path.join(src_dir+curdirprofile, "*.jpg")):
            jpgfile =  jpgfile.split('\\')[0]+"/" +jpgfile.split('\\')[1]
            #im = Image.open(jpgfile).convert("RGB")
            #stat = ImageStat.Stat(im)
            #if sum(stat.sum)/3==stat.sum[0]:
            #    print(jpgfile)
            if a<2:
                profileFile.write(jpgfile+"\n")
            elif (a < 5 and random()<0.6):
                profileFile.write(jpgfile+"\n")
            else:
                break
            a += 1
        

In [18]:
def png_to_jpg(src_dir3):
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.png")):
        name = jpgfile.split('\\')[1][:-3]
        
        jpgfile =  jpgfile.split('\\')[0]+"/" +jpgfile.split('\\')[1]
        im = Image.open(jpgfile)
        rgb = im.convert('RGB')
        rgb.save(name+"jpg")
       

In [19]:

src_dir = "C:/Users/d.baranovska/Downloads/combined_data"
src_dir_front = "./TPdata/train_data/front"
src_dir_profile = "./TPdata/train_data/profile"
cfp_data = "C:/Users/d.baranovska/Downloads/cfp_data/Images"
relocate_data(src_dir, src_dir_front, src_dir_profile)
choose_cfp(cfp_data, src_dir_front, src_dir_profile)
frontFile.close()
profileFile.close()

In [20]:
frontFile = open("./TPdata/train_data/front_list.txt", "r+")
profileFile = open("./TPdata/train_data/profile_list.txt", "r+")
#x.split('\\')[0]+"/" #+x.split('\\')[1].strip()
front_list = list(map(lambda x: x.strip(), frontFile.readlines()))
profile_list = list(map(lambda x: x.strip(),profileFile.readlines()))
frontFile.close()
profileFile.close()

In [21]:
#front_list[0].strip('\\')

In [22]:
print(len(front_list))
print(len(profile_list))

2018
2243


In [23]:
PIC_SIZE = (224, 224)
class TrainDataset(Dataset):
    def __init__(self, img_list):
        super(type(self), self).__init__()
        self.img_list = img_list
    def __len__(self):
        return len(self.img_list)
    def __getitem__(self, idx):
        #229_01_01_200_08_cropped_test.png
        img_name = self.img_list[idx]
        with Image.open(img_name) as i:
            width, height = i.size
            if width != height:
                sz = min(width, height)
                left = (width - sz)/2
                right = (width + sz)/2
                top = (height - sz)/2
                bottom = (height + sz)/2
                i = i.crop((left, top, right, bottom))
            im = i.resize(PIC_SIZE)
            im = transforms.RandomHorizontalFlip()(im)
            our_tensor = transforms.ToTensor()(im)
            #print(our_tensor.size(), self.img_list[idx])
        return our_tensor

In [24]:
# #Test set

# test_list = []
# src_test_data = "C:/Users/d.baranovska/Git/FNM-with-Extreme-Pose-and-Expression-in-the-Wild/TPdata/test_data"
# for x in glob.iglob(os.path.join(src_test_data+"/test", "*.jpg")):
#     jpgfile = x.split('\\')[0]+"/" +x.split('\\')[1].strip()
#     test_list.append(jpgfile)
# Test_data = TrainDataset(test_list)
# test_loader = torch.utils.data.DataLoader(dataset=Test_data, batch_size=BS, shuffle=True, drop_last=True)
# enumerate_test = list(enumerate(test_loader))
# for j in range(len(enumerate_test)):
#         enumerate_test[j] = (j, torch.autograd.Variable(enumerate_test[j][1].to(device), requires_grad = False))
   

# Generator 

In [25]:
f7_shape = [7, 7, 2048]

#### second variant

In [26]:
class Generator(nn.Module):
    def __init__(self): #, profile, front, train):
        super(Generator, self).__init__() 
       
        self.conv1 = nn.Sequential(conv2d(2048, 512, kernel_size=1, strides = 1), NORM(512, is_train), nn.ReLU())
    
        self.res1_1 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.res1_2 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.res1_3 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.res1_4 = res_block(512, 512, norm = NORM, pad_input = 7)
        self.dconv2 = nn.Sequential(deconv2d(512, 256, kernel_size=4, strides = 2, pad_input = 7), NORM(256), nn.ReLU())
        self.res2 = res_block(256, 256, norm = NORM, pad_input = 14)
        self.dconv3 = nn.Sequential(deconv2d(256, 128, kernel_size=4, strides = 2, pad_input = 14) , NORM(128), nn.ReLU())
        self.res3 = res_block(128, 128, norm = NORM, pad_input = 28)
        self.dconv4 = nn.Sequential(deconv2d(128, 64, kernel_size=4, strides = 2, pad_input = 28), NORM(64), nn.ReLU())
        self.res4 = res_block(64, 64, norm = NORM, pad_input = 56)
        self.dconv5 = nn.Sequential(deconv2d(64, 32, kernel_size=4, strides = 2, pad_input = 56), NORM(32), nn.ReLU())
        self.res5 = res_block(32, 32, norm = NORM, pad_input = 112)
        self.dconv6 = nn.Sequential(deconv2d(32, 32, kernel_size=4, strides = 2, pad_input =112),  NORM(32), nn.ReLU())
        self.res6 = res_block(32, 32, norm = NORM,  pad_input = 224)
        self.gen = nn.Sequential(conv2d(32, 3,  kernel_size=1, strides = 1), nn.Tanh())
        
    def forward(self, feature):
        featt7 = feature[0]
        feat7 = self.conv1(featt7)
        pool5 = feature[1]
        res1_1 = self.res1_1(feat7)
        res1_2 = self.res1_2(res1_1)
        res1_3 = self.res1_3(res1_2)
        res1_4 = self.res1_2(res1_3)
        dconv2 = self.dconv2(res1_4)
        res2 = self.res2(dconv2)
        dconv3 = self.dconv3(res2)
        res3 = self.res3(dconv3)
        dconv4 = self.dconv4(res3)
        res4 = self.res4(dconv4)
        dconv5 = self.dconv5(res4)
        res5 = self.res5(dconv5)
        dconv6 = self.dconv6(res5)
        res6 = self.res6(dconv6)
        gen = self.gen(res6)
        return gen

In [27]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        pad = 1
        self.h0_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h0_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h0_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h0_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h0_4 = nn.Sequential(conv2d(256, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h0_5 = nn.Linear(12544, 1)
        
        self.h1_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h1_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h1_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h1_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h1_4 = nn.Linear(6144, 1)
        
        self.h2_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h2_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h2_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h2_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        #self.h2_4 = nn.Linear(self.h2_3.out_features, 1)
        self.h2_4 = nn.Linear(3840, 1)
        
        self.h3_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h3_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h3_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h3_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h3_4 = nn.Linear(2560, 1)
        
        self.h4_0 = nn.Sequential(conv2d(3, 32, kernel_size=3, strides=2, padding = pad), nn.LeakyReLU())
        self.h4_1 = nn.Sequential(conv2d(32, 64, kernel_size=3, strides=2, padding = pad), NORM(64), nn.LeakyReLU())
        self.h4_2 = nn.Sequential(conv2d(64, 128, kernel_size=3, strides=2, padding = pad), NORM(128), nn.LeakyReLU())
        self.h4_3 = nn.Sequential(conv2d(128, 256, kernel_size=3, strides=2, padding = pad), NORM(256), nn.LeakyReLU())
        self.h4_4 = nn.Linear(16384, 1)
    
    
    def forward(self, images):
        eyes = images[0:BS,0:CHANNEL,64:100,50:174] #[BS,36,124,CHANNEL])
        nose =  images[0:BS,0:CHANNEL,75:140,90:134]#tf.slice(images, [0,75,90,0], [BS,65,44,CHANNEL])
        mouth = images[0:BS,0:CHANNEL,140:170,75:149]#tf.slice(images, [0,140,75,0], [BS,30,74,CHANNEL])
        face = images[0:BS,0:CHANNEL,64:180,50:174]#[BS,116,124,CHANNEL])
        h0_0 = self.h0_0(images)
        h0_1 = self.h0_1(h0_0)
        h0_2 = self.h0_2(h0_1)
        h0_3 = self.h0_3(h0_2)
        h0_4 = self.h0_4(h0_3)
        h0_5 = self.h0_5(torch.reshape(h0_4, [BS, -1]))
        
        h1_0 = self.h1_0(eyes)
        h1_1 = self.h1_1(h1_0)
        h1_2 = self.h1_2(h1_1)
        h1_3 = self.h1_3(h1_2)
        h1_4 = self.h1_4(torch.reshape(h1_3, [BS, -1]))
        
        h2_0 = self.h2_0(nose)
        h2_1 = self.h2_1(h2_0)
        h2_2 = self.h2_2(h2_1)
        h2_3 = self.h2_3(h2_2)
        h2_4 = self.h2_4(torch.reshape(h2_3, [BS, -1]))
        
        h3_0 = self.h3_0(mouth)
        h3_1 = self.h3_1(h3_0)
        h3_2 = self.h3_2(h3_1)
        h3_3 = self.h3_3(h3_2)
        h3_4 = self.h3_4(torch.reshape(h3_3, [BS, -1]))
        
        h4_0 = self.h4_0(face)
        h4_1 = self.h4_1(h4_0)
        h4_2 = self.h4_2(h4_1)
        h4_3 = self.h4_3(h4_2)
        h4_4 = self.h4_4(torch.reshape(h4_3, [BS, -1]))
        return torch.cat((h0_5, h1_4, h2_4, h3_4, h4_4))

# G and D

In [28]:
#built network
#z_dim = 224*224
#mnist_dim = 224*224
#mnist_dim = train_dataset_front.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator().to(device)
D = Discriminator().to(device)

In [29]:
#print(torch.__version__)

In [30]:
criterion = nn.MSELoss(reduction='mean')

# optimizer
lr = 1e-4 
G_optimizer = optim.Adam(G.parameters(), lr,betas=(beta1, beta2), weight_decay=1e-5)
D_optimizer = optim.Adam(D.parameters(), lr, betas=(beta1, beta2), weight_decay=1e-5)


# Loss functions

In [31]:
# Losses = []
# D_finalLosses, G_finalLosses = [], []

In [32]:
# def reverse(x, dim):
#     dim = x.dim() + dim if dim < 0 else dim
#     return x[tuple(slice(None, None) if i != dim
#              else torch.arange(x.size(i)-1, -1, -1).long()
#              for i in range(x.dim()))]

In [33]:
epsilon = 1e-9

In [34]:
global a 
a = 0
global epoch_num
epoch_num = 0

In [35]:

#print("Gen.gen parametrs", torch.sum(i(*G.parameters())).item())

In [36]:
def loss(epoch, profile, front):
    #=======================Train the generator=======================#
    G_optimizer.zero_grad()
    D_optimizer.zero_grad()
    Model.eval()
    for p in Model.parameters():
        p.require_grad = False
    feature_p = Model.forward(profile) # G_enc(x)
    feature_f = Model.forward(front) # G_enc(y)
#     feature_p = [i for i in feature_p]
#     feature_f = [i for i in feature_f]
#     for i in feature_p:
#         i = i.to(device)
#     for i in feature_f:
#         i = i.to(device)
    gen_p = G.forward(feature_p) # ~x
    gen_f = G.forward(feature_f) # ~y
    if a % 200 == 0:
#        test_pic = Model(enumerate_test[0][1])
#         gen_test = G(test_pic)
#         save_image(gen_test, src_test_data+"/result/"+str(epoch)+"_"+str(a)+"_"+".png")

    
    
        print('')
        print(a, epoch)
        print("gen_p 0 - 1", torch.sum(gen_p[0] -  gen_p[1]))
        print("front 0 - 1", torch.sum(front[0] -  front[1]))
        #print("Model conv5_3_3x3 sum =", torch.sum(*Model.conv5_3_3x3.parameters()).item())
        #print("Gen.dconv6 parameters sum", torch.sum(*G.dconv6.parameters()).item())
        print("Gen.gen params sum", torch.sum(*G.gen.parameters()).item())
        #print("Dis.h0_5 params sum", torch.sum(*D.h0_5.parameters()).item())
        #save_image(feature_p[0].int(), "./TPdata/train_data/processed/" + str(a)+ "_feature_p" + ".png")
        save_image(profile, "I:/train_data/processed/epoch" + str(epoch_num)+"_"+ str(a)+"_profile" + ".png")
        save_image(gen_f, "I:/train_data/processed/epoch" + str(epoch_num)+ "_"+str(a)+ "_gen_f_all" + ".png")
        save_image(gen_p, "I:/train_data/processed/epoch" +str(epoch_num)+ "_"+str(a)+  "_gen_p_" + ".png")
    #for i in gen_p: writer.add_image('~x - gen_p', i)
    #for i in gen_f: writer.add_image('~y - gen_f', i)
#     dr = torch.cat(D(front))
#     df1 = torch.cat(D(gen_p))
#     df2 = torch.cat(D(gen_f))
    dr = D(front)
    df1 = D(gen_p)
    df2 = D(gen_f)
    feature_gen_p = Model.forward(gen_p) #G_enc(~x)
    feature_gen_f = Model.forward(gen_f) #G_enc(~y)
    
    pool5_p_norm = feature_p[-1]/(torch.norm(feature_p[-1],dim = 1, keepdim = True) + epsilon)
    pool5_f_norm = feature_f[-1]/(torch.norm(feature_f[-1],dim = 1, keepdim = True) + epsilon)
    
    pool5_gen_p_norm = feature_gen_p[-1]/(torch.norm(feature_gen_p[-1],dim = 1, keepdim = True) + epsilon)
    pool5_gen_f_norm = feature_gen_f[-1]/(torch.norm(feature_gen_f[-1],dim = 1, keepdim = True) + epsilon)
    
     # 1. Frontalization Loss: L1-Norm
    front_loss = torch.mean(torch.sum(torch.abs(front/255. - gen_f/255.), [1,2,3]))
    
    # 2. identity perseption loss l2-norm
    feature_distance = 0.5*(1 - torch.sum(torch.mul(pool5_p_norm, pool5_gen_p_norm), [1])) + \
                                0.5*(1 - torch.sum(torch.mul(pool5_f_norm, pool5_gen_f_norm), [1]))
    feature_loss = torch.mean(feature_distance)
    #Losses.append(feature_loss.item())
    #.
    #trainable var
    #all_vars = torch.autograd.Variables()
    #vars_gen = Variable(G.parameters, requires_grad= True)
    #vars_dis = Variable(D.parameters, requires_grad= True)
    #vars_gen = weights_list=[var for var in self.vars_gen if 'kernel' in var.name]
    
    
    # 3. Regulation loss
    #reg_gen = 0.5*criterion(gen_p, profile)+ 0.5*criterion(gen_p, front)
    #reg_gen = criterion(gen_p, profile)*0.9991 + criterion(gen_p, front)* 0.0009
    #reg_dis = criterion(gen_f, front)
    #reg_gen =  torch.optim.SGD(G.parameters(), lr = lambda_reg)
    #reg_dis =  torch.optim.SGD(D.parameters(), lr = lambda_reg)
#     loss = nn.MSELoss(lambda_reg, reduction='mean')
#     #reg_gen = loss(weights_list=[var for var in vars_gen]) # if 'kernel' in var.name
#     #reg_dis = loss(weights_list=[var for var in vars_dis])# if 'kernel' in var.name
#     target = torch.zeros(G.parameters.shape)
#     reg_gen = loss(G.parameters, target)
#     reg_dis = loss(D.parameters, target)
#     G_losses.append(reg_gen)
#     D_losses.append(reg_dis)
    
    
    # 4. Adversarial Loss
    #d_loss = torch.mean(torch.sum(df1)*0.9 + torch.sum(df2)*0.1 - torch.sum(dr)) / 5
    #g_loss = -torch.mean(torch.sum(df1)*0.9 + torch.sum(df2)*0.1) / 5
    d_loss = torch.mean(torch.sum(df1)*0.5 + torch.sum(df2)*0.5 - torch.sum(dr)) / 5
    g_loss = -torch.mean(torch.sum(df1)*0.5 + torch.sum(df2)*0.5) / 5
    
    
    #D_losses.append(d_loss.item())
    #G_losses.append(g_loss.item())
    
    # 5. Symmetric Loss - not applied
#     mirror_p = torch.flip(gen_p, dims=[3])
#     sym_loss = torch.mean(torch.sum(torch.abs(mirror_p/225. - gen_p/255.), [1,2,3]))
    
    # Gradient Penalty #
#     alpha = torch.rand([front.size(0), 1, 1, 1]).to(device)
#     inter = (alpha*front + ((1 - alpha) * gen_p )).requires_grad_(True).to(device)
#     d = D(inter)
#     fake = Variable(torch.Tensor(BS*5, 1).fill_(1.0), requires_grad=False).to(device)
#     grad = torch.autograd.grad(d, inter, grad_outputs=fake,
#     retain_graph=True,
#     create_graph=False,
#     only_inputs=True,)[0]
#     #grad = d.grad
#     grad = grad.view(grad.size(0), -1)
#     gradient_penalty = ((grad.norm(2, dim=1)-1)**2).mean()
      
     # 6. Drift Loss - not applied
    #drift_loss = 0
    #torch.mean(torch.add(torch.square(df)) + torch.add(torch.square(dr))) / 10
    
    
    
    Gen_loss = lambda_l1*front_loss + lambda_fea*feature_loss +(lambda_gan)*g_loss# + reg_gen#+ 0.0009*sym_loss #
    #if epoch %17 ==0:
     #   Gen_loss += 0.001*sym_loss
    Dis_loss = lambda_gan* d_loss #+ reg_dis#+lambda_gp*gradient_penalty# + torch.tensor(lambda_gp) * gradient_penalty
   
    
    #Gen_loss = Gen_loss.to(device)
    #Dis_loss = Gen_loss.to(device)
    #G_finalLosses.append(Gen_loss.item())
    #D_finalLosses.append(Dis_loss.item())
#     Gen_loss.backward()
#     Dis_loss.backward()
    
    (Gen_loss + Dis_loss).backward()
    

    
    G_optimizer.step()
    D_optimizer.step()
    G_optimizer.zero_grad()
    D_optimizer.zero_grad()
#     if a < 30:
#         writer.add_scalar('Gen_loss', Gen_loss , epoch_num*30 + a)
#         writer.add_scalar('Dis_loss', Dis_loss , epoch_num*30 + a)
#         writer.add_scalar('front_loss',  front_loss , epoch_num*30 + a)
#         writer.add_scalar('feature_loss',  feature_loss , epoch_num*30 + a)
#         writer.add_scalar('g_loss',  g_loss , epoch_num*30 + a)
#         writer.add_scalar('d_loss',  d_loss , epoch_num*30 + a)
#         writer.add_scalar(' gradient_penalty',gradient_penalty, epoch_num*30 + a )
#         writer.add_image('gen_p', gen_p[0], epoch_num*100 + a )
#         writer.add_image('gen_f', gen_f[0], epoch_num*100 + a)
#         writer.add_image('front',  front[0], epoch_num*100 + a)
#         writer.add_image('profile',  profile[0], epoch_num*100 + a)
#         writer.add_image('gen_p', gen_p[1], epoch_num*100 + a)
#         writer.add_image('gen_f',  gen_f[1], epoch_num*100 + a)
#         writer.add_image('front', front[1], epoch_num*100 + a)
#         writer.add_image('profile',  profile[1], epoch_num*100 + a)
#         writer.add_image('gen_p', gen_p[2], epoch_num*100 + a)
#         writer.add_image('gen_f', gen_f[2], epoch_num*100 + a)
#         writer.add_image('front', front[2], epoch_num*100 + a)
#         writer.add_image('profile',  profile[2], epoch_num*100 + a)

    
    return Gen_loss,  Dis_loss, front_loss, feature_loss, g_loss, d_loss #, sym_loss#, reg_gen,reg_dis
    
#     z = Variable(torch.randn(bs, z_dim).to(device))
#     y = Variable(torch.ones(bs, 1).to(device))

#     G_output = G(z)
#     D_output = D(G_output)
#     G_loss = criterion(D_output, y)

#     # gradient backprop & optimize ONLY G's parameters
#     G_loss.backward()
#     G_optimizer.step()
        
#     return G_loss.data.item()

In [37]:
sum(p.numel() for p in G.parameters() if p.requires_grad)


24322144

In [38]:
# Model conv5_3_3x3 sum = -1227.2626953125

# Main

In [39]:
# x = torch.rand([BS, 3, 224, 224])
# y = torch.rand([BS, 3, 224, 224])
# t = Model(x.to(device))
# x_gen = G(t)
# #reg_gen = criterion(x_gen, x)
# #print(reg_gen)
# print(x_gen.size())
# print("Model conv5_3_3x3 sum =", torch.sum(*Model.conv5_3_3x3.parameters()).item())

In [40]:
# x_dis = D(x_gen)
# print(x_dis[0].size(), x_dis[1].size(), x_dis[2].size(), x_dis[3].size(), x_dis[4].size())

In [41]:
Train_front_data = TrainDataset(front_list)
Train_profile_data = TrainDataset(profile_list)

In [42]:
train_loader_front = torch.utils.data.DataLoader(dataset=Train_front_data, batch_size=BS, shuffle=True, drop_last=True, num_workers = 0)
train_loader_profile = torch.utils.data.DataLoader(dataset=Train_profile_data, batch_size=BS, shuffle=True, drop_last=True, num_workers = 0)
#test_loader = torch.utils.data.DataLoader(dataset=Test_dataset, batch_size=bs, shuffle=False)

In [43]:
d = list(enumerate(train_loader_front))
print(d[3][1].size())

torch.Size([4, 3, 224, 224])


In [44]:
(profile_list[0].split('\\')[-1].split('.')[0])

'C:/Users/d'

In [ ]:
n_epoch = 300
Gen_loss=0
Dis_loss=0
D_losses, G_losses = [], []
for epoch in range(1, n_epoch+1):
    #if not os.path.isdir("I:/train_data/processed/epoch" + str(epoch_num)):
    #    os.mkdir( "I:/train_data/processed/epoch" + str(epoch_num))
    #os.mkdir( "I:/train_data/processed/epoch" + str(epoch_num))

    d = list(enumerate(train_loader_front))
    for j in range(len(d)):
        d[j] = (j, torch.autograd.Variable(d[j][1].to(device), requires_grad = False))
    for batch_idx, x in enumerate(train_loader_profile):
        for k in range(len(x)):
            x[k] = torch.autograd.Variable(x[k].to(device), requires_grad = False)
        x = x.to(device)
        if batch_idx >= len(d):
            break
        Gen_loss,  Dis_loss, front_loss, feature_loss, g_loss, d_loss= loss(epoch, x, d[batch_idx][1])
        #, reg_gen,reg_dis
        
        
        a += 1
    a = 0
    epoch_num += 1 
        #print(list(map(lambda x: x.item(), [Gen_loss,  Dis_loss, front_loss, feature_loss, g_loss, d_loss])))
#, reg_gen: %.3f,reg_dis: %.3f
#     for batch_idx_test in range(len(enumerate_test)):
#         feeature_p = Model.forward(enumerate_test[batch_idx_test][1]) 
#         geen_p = G.forward(feeature_p) # ~x
#         save_image(geen_p, src_test_data+"/result/"+str(epoch)+"_"+str(batch_idx_test)+".jpg" )
#     #model_weights_dir = "C:/Users/d.baranovska/Git/FNM-with-Extreme-Pose-and-Expression-in-the-Wild/weights"
    #torch.save(G.state_dict(), model_weights_dir+"/G.pth")
    #torch.save(D.state_dict(), model_weights_dir+"/D.pth")
    D_losses.append(Gen_loss)
    G_losses.append(Dis_loss)
    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f,\
          front_loss: %.3f, feature_loss: %.3f, g_loss: %.3f, d_loss: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses)),
        front_loss, feature_loss, g_loss, d_loss))#,  reg_gen,reg_dis))
    if(epoch % 20 == 0):
        plt.plot(G_losses)
        plt.show()
        
    #writer.add_histogram('Gen_losses', np.array(G_losses))
    #writer.add_histogram('Dis_losses', np.array(D_losses))


0 1
gen_p 0 - 1 tensor(-175.7921, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(21170.3301, device='cuda:0')
Gen.gen params sum 0.3989993929862976

200 1
gen_p 0 - 1 tensor(-248.3670, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-9818.1055, device='cuda:0')
Gen.gen params sum 1.0582157373428345

400 1
gen_p 0 - 1 tensor(5074.8027, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(1761.2825, device='cuda:0')
Gen.gen params sum 1.1011157035827637
[1/300]: loss_d: -888.802, loss_g: -3.181,          front_loss: 106.026, feature_loss: 0.000, g_loss: -888.914, d_loss: -3.181

0 2
gen_p 0 - 1 tensor(-7005.6797, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(3937.1768, device='cuda:0')
Gen.gen params sum 1.123945951461792

200 2
gen_p 0 - 1 tensor(-9534.2910, device='cuda:0', grad_fn=<SumBackward0>)
front 0 - 1 tensor(-13499.1836, device='cuda:0')
Gen.gen params sum 1.1753228902816772

400 2
gen_p 0 - 1 tensor(-16732.2852, device='cuda:0', g

In [ ]:
plt.plot(D_losses)
plt.show()

In [ ]:
model_weights_dir = "C:/Users/d.baranovska/Git/FNM-with-Extreme-Pose-and-Expression-in-the-Wild/weights"
torch.save(G.state_dict(), model_weights_dir+"/G.pth")
torch.save(D.state_dict(), model_weights_dir+"/D.pth")

In [ ]:
def fnm_gen(weights_path=None, **kwargs):
    """
    load imported model instance

    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Generator()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

def fnm_dis(weights_path=None, **kwargs):
    """
    load imported model instance

    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Discriminator()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [ ]:
# #Test set
# test_list = []
# src_test_data = "C:/Users/d.baranovska/Git/FNM-with-Extreme-Pose-and-Expression-in-the-Wild/TPdata/test_data"
# for x in glob.iglob(os.path.join(src_test_data+"/Not_tested", "*.jpg")):
#     jpgfile = x.split('\\')[0]+"/" +x.split('\\')[1].strip()
#     test_list.append(jpgfile)
# Test_data = TrainDataset(test_list)
# test_loader = torch.utils.data.DataLoader(dataset=Test_data, batch_size=BS, shuffle=True, drop_last=True)
# enumerate_test = list(enumerate(test_loader))
# for j in range(len(enumerate_test)):
#         enumerate_test[j] = (j, torch.autograd.Variable(enumerate_test[j][1].to(device), requires_grad = False))
# for batch_idx_test in range(len(enumerate_test)):
#         feeature_p = Model.forward(enumerate_test[batch_idx_test][1]) 
#         geen_p = G.forward(feeature_p) # ~x
#         save_image(geen_p, src_test_data+"/Not_tested/results/"+"_"+str(batch_idx_test)+".png" )
#         save_image(enumerate_test[batch_idx_test][1], src_test_data+"/Not_tested/results/"+"_"+str(batch_idx_test)+"orig.png" )
    

In [ ]:
# #Test set
# test_list = []
# src_test_data = "C:/Users/d.baranovska/Git/FNM-with-Extreme-Pose-and-Expression-in-the-Wild/TPdata/test_data"
# for x in glob.iglob(os.path.join(src_test_data+"/test", "*.jpg")):
#     jpgfile = x.split('\\')[0]+"/" +x.split('\\')[1].strip()
#     test_list.append(jpgfile)
# Test_data = TrainDataset(test_list)
# test_loader = torch.utils.data.DataLoader(dataset=Test_data, batch_size=BS, shuffle=True, drop_last=True)
# enumerate_test = list(enumerate(test_loader))
# for j in range(len(enumerate_test)):
#         enumerate_test[j] = (j, torch.autograd.Variable(enumerate_test[j][1].to(device), requires_grad = False))
# for batch_idx_test in range(len(enumerate_test)):
#         feeature_p = Model.forward(enumerate_test[batch_idx_test][1]) 
#         geen_p = G.forward(feeature_p) # ~x
#         save_image(geen_p, src_test_data+"/test/results/"+"_"+str(batch_idx_test)+".png" )
#         save_image(enumerate_test[batch_idx_test][1], src_test_data+"/test/results/"+"_orig"+str(batch_idx_test)+".png" )
    